In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
import pickle as pkl
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
#import decoupler
print(sys.executable)
import gseapy
from gseapy.plot import barplot, dotplot
warnings.filterwarnings("ignore")
import re
from matplotlib.colors import Normalize

In [ ]:
pre = "MH123"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)
def remove_go(term):
    # Use regular expression to find and replace GO terms, ensuring the string is treated as a raw string
    new_term = re.sub(r'\s*\(GO:\d+\)', '', term)  # Corrected regex finds the GO term pattern and removes it
    return new_term.strip() 

col_human =  "#e35e28"
col_cyno ="#31c7ba"
_,base_table_path_MH115,_,_ = h.return_local_paths(drive = drive,pre = 'MH115',add_path = True)
_,base_table_path_MH122,_,_ = h.return_local_paths(drive = drive,pre = 'MH122',add_path = True)

In [ ]:
celltypes_cyno_DGE = ['CD4_T', 'CD14_Mono', 'NKProliferating', 'CD8_T', 'B', 'MAIT', 'dnT']
celltypes_human_DGE = ['CD4_T', 'CD14_Mono', 'NKProliferating', 'CD8_T', 'B', 'MAIT']
conditions  = ['timepoints06hr','timepoints24hr']

In [ ]:
unfiltered_pathways_DGE = []
for celltype in celltypes_cyno_DGE:
    for condition in conditions:
        df_all_df = pd.read_csv(os.path.join(base_table_path_MH115,
                                             'MH115' + '_df_pathways_' + str(celltype) + '_' + str(condition) + '_' +'cyno' + '.csv'),
                                index_col=0)
        unfiltered_pathways_DGE.append(df_all_df)

for celltype in celltypes_human_DGE:
    for condition in conditions:
        df_all_df = pd.read_csv(os.path.join(base_table_path_MH115,
                                             'MH115' + '_df_pathways_' + str(celltype) + '_' + str(condition) + '_' +'human' + '.csv'),
                                index_col=0)
        unfiltered_pathways_DGE.append(df_all_df)

In [ ]:
df_unfiltered_pathways_DGE = pd.concat(unfiltered_pathways_DGE,ignore_index=True)
df_unfiltered_pathways_DGE['Log Odds Ratio'] = np.log(df_unfiltered_pathways_DGE['Odds Ratio'])
df_unfiltered_pathways_DGE['celltype'] = df_unfiltered_pathways_DGE['celltype'].replace({
    'NKProliferating': 'NK+Proliferating',
    'CD14_Mono': 'CD14 Mono',
    'CD4_T': 'CD4 T',
    'CD8_T': 'CD8 T'
})
df_unfiltered_pathways_DGE.to_csv(os.path.join(base_table_path,pre + '_df_unfiltered_pathways_DGE.csv'))

In [ ]:
#df_all_pathways = pd.read_csv(os.path.join(base_table_path_MH115,'MH115_all_df_pathways_.csv'),index_col=0)

In [ ]:
df_all_pathways_VAE = pd.read_csv(os.path.join(base_table_path_MH122,'MH122_df_all_pathways_VAE.csv'),index_col=0)

In [ ]:
df_all_pathways_VAE_ct = df_all_pathways_VAE[df_all_pathways_VAE['celltype'] == celltype]

In [ ]:
pathway_comp_VAE_DGE = []
for celltype in ['B', 'CD14 Mono', 'CD16 Mono', 'CD4 T', 'CD8 T', 'MAIT','NK+Proliferating']:
    df_all_pathways_VAE_ct = df_all_pathways_VAE[df_all_pathways_VAE['celltype'] == celltype]
    #df_all_pathways_VAE_ct = df_all_pathways_VAE_ct.drop_duplicates(subset=['Genes'], keep='first')
    df_all_pathways_VAE_ct_top5 = df_all_pathways_VAE_ct.head(5)
    df_all_pathways_VAE_ct_top5 = df_all_pathways_VAE_ct_top5.rename(columns={
        'Genes': 'Genes_VAE',
        'Log Odds Ratio': 'Log Odds Ratio_VAE',
        'Adjusted P-value': 'Adjusted P-value_VAE',
        'short_term': 'short_term_VAE'
    })
    for species in ['human','cyno']:
        for condition in ['timepoints06hr','timepoints24hr']:
            # Filter df_all_pathways to only include rows matching specific species and comparison
            filtered_df_all_pathways = df_unfiltered_pathways_DGE[
                (df_unfiltered_pathways_DGE['species'] == species) &
                (df_unfiltered_pathways_DGE['comparison.vs.00hr'] == condition) &
                (df_unfiltered_pathways_DGE['celltype'] == celltype)
            ]
            
            df_all_pathways_VAE_ct_top5 = pd.merge(
                df_all_pathways_VAE_ct_top5,
                filtered_df_all_pathways[['Genes','Term', 'Log Odds Ratio','Adjusted P-value']],  # Include only necessary columns
                on='Term',
                how='left'
            )
            
            df_all_pathways_VAE_ct_top5 = df_all_pathways_VAE_ct_top5.rename(columns={
                'Genes': 'Genes_'+species + '_'+condition[-4:],
                'Log Odds Ratio': 'Log Odds Ratio_' + species + '_'+condition[-4:],
                'Adjusted P-value': 'Adjusted P-value_' + species + '_'+condition[-4:]
            })
    pathway_comp_VAE_DGE.append(df_all_pathways_VAE_ct_top5)
df_pathway_comp_VAE_DGE=pd.concat(pathway_comp_VAE_DGE,ignore_index=True)

In [ ]:
df_pathway_comp_VAE_DGE=pd.concat(pathway_comp_VAE_DGE,ignore_index=True)
df_pathway_comp_VAE_DGE.to_csv(os.path.join(base_table_path,pre + '_df_pathway_comp_VAE_DGE.csv'))

In [ ]:
columns_heatmap = ['celltype','short_term_VAE','Log Odds Ratio_cyno_06hr','Log Odds Ratio_human_06hr','Log Odds Ratio_cyno_24hr','Log Odds Ratio_human_24hr']

In [ ]:
heatmap_data_all = df_pathway_comp_VAE_DGE[columns_heatmap]
heatmap_data_all.set_index('short_term_VAE',inplace=True)

In [ ]:
plt.figure(figsize=(7, 10))
norm = Normalize(vmin=0, vmax=5)

# Prepare the heatmap without the 'celltype' column
heatmap_ax = sns.heatmap(heatmap_data_all.drop('celltype', axis=1), cmap='Reds', annot=False, linewidths=.8, linecolor='black',norm=norm)

# Set the tick rotation
plt.xticks(rotation=90)
plt.yticks(rotation=0)

# Set y-tick labels to be bold
plt.gca().set_yticklabels(plt.gca().get_yticklabels(), fontweight='bold')

# Create a color bar with a title
cbar = heatmap_ax.collections[0].colorbar
cbar.set_label('log odds ratio',labelpad=10)  
plt.savefig(os.path.join(base_plots_path,pre + '_heatmap_pathways.pdf'),bbox_inches='tight')
plt.show()

save session

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()